In [4]:
from __future__ import print_function

import warnings

warnings.filterwarnings('ignore')

import argparse
import torch
import torch.utils.data
from torch.utils.data import DataLoader, TensorDataset
from torch import nn, optim
from torch.nn import functional as F

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import get_data, utils

In [6]:
# Set up experiment parameters
params = utils.Params("model_configurations/experiment_params.json")
X, y, cols = get_data.get_and_preprocess_german(params)

features = [c for c in X]

gender_indc = features.index('Gender')
loan_rate_indc = features.index('LoanRateAsPercentOfIncome')

X = X.values

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame(X)

In [14]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1,0,1,67,6,1169,4,4,2,1,...,0,0,0,0,1,0,0,0,1,1
1,0,0,0,22,48,5951,2,2,1,1,...,0,0,0,0,1,0,0,0,0,1
2,1,0,1,49,12,2096,2,3,1,2,...,0,0,0,0,1,0,0,0,1,0
3,1,0,1,45,42,7882,2,4,1,2,...,0,0,0,1,0,0,0,0,1,1
4,1,0,1,53,24,4870,3,4,2,2,...,0,0,0,0,0,0,0,0,0,1


In [16]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.00000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,0.690000,0.037000,0.54800,35.546000,20.903000,3271.258000,2.973000,2.845000,1.407000,1.155000,...,0.13900,0.0,0.041000,0.052000,0.713000,0.179000,0.062000,0.172000,0.42700,0.778000
std,0.462725,0.188856,0.49794,11.375469,12.058814,2822.736876,1.118715,1.103718,0.577654,0.362086,...,0.34612,0.0,0.198389,0.222138,0.452588,0.383544,0.241276,0.377569,0.49489,0.415799
min,0.000000,0.000000,0.00000,19.000000,4.000000,250.000000,1.000000,1.000000,1.000000,1.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.00000,27.000000,12.000000,1365.500000,2.000000,2.000000,1.000000,1.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000
50%,1.000000,0.000000,1.00000,33.000000,18.000000,2319.500000,3.000000,3.000000,1.000000,1.000000,...,0.00000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,1.000000
75%,1.000000,0.000000,1.00000,42.000000,24.000000,3972.250000,4.000000,4.000000,2.000000,1.000000,...,0.00000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.00000,1.000000
max,1.000000,1.000000,1.00000,75.000000,72.000000,18424.000000,4.000000,4.000000,4.000000,2.000000,...,1.00000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [21]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
df_scaled = pd.DataFrame(X)

In [22]:
df_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1.0,0.0,1.0,0.857143,0.029412,0.050567,1.000000,1.000000,0.333333,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,0.0,0.0,0.0,0.053571,0.647059,0.313690,0.333333,0.333333,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,1.0,0.535714,0.117647,0.101574,0.333333,0.666667,0.000000,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.464286,0.558824,0.419941,0.333333,1.000000,0.000000,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4,1.0,0.0,1.0,0.607143,0.294118,0.254209,0.666667,1.000000,0.333333,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
cols = "Gender,ForeignWorker,Single,Age,LoanDuration,LoanAmount,LoanRateAsPercentOfIncome,YearsAtCurrentHome,NumberOfOtherLoansAtBank,NumberOfLiableIndividuals,HasTelephone,CheckingAccountBalance_geq_0,CheckingAccountBalance_geq_200,SavingsAccountBalance_geq_100,SavingsAccountBalance_geq_500,MissedPayments,NoCurrentLoan,CriticalAccountOrLoansElsewhere,OtherLoansAtBank,OtherLoansAtStore,HasCoapplicant,HasGuarantor,OwnsHouse,RentsHouse,Unemployed,YearsAtCurrentJob_lt_1,YearsAtCurrentJob_geq_4,JobClassIsSkilled".split(",")

['Gender',
 'ForeignWorker',
 'Single',
 'Age',
 'LoanDuration',
 'LoanAmount',
 'LoanRateAsPercentOfIncome',
 'YearsAtCurrentHome',
 'NumberOfOtherLoansAtBank',
 'NumberOfLiableIndividuals',
 'HasTelephone',
 'CheckingAccountBalance_geq_0',
 'CheckingAccountBalance_geq_200',
 'SavingsAccountBalance_geq_100',
 'SavingsAccountBalance_geq_500',
 'MissedPayments',
 'NoCurrentLoan',
 'CriticalAccountOrLoansElsewhere',
 'OtherLoansAtBank',
 'OtherLoansAtStore',
 'HasCoapplicant',
 'HasGuarantor',
 'OwnsHouse',
 'RentsHouse',
 'Unemployed',
 'YearsAtCurrentJob_lt_1',
 'YearsAtCurrentJob_geq_4',
 'JobClassIsSkilled']

In [26]:
df.columns = cols
df.head()

,Gender,ForeignWorker,Single,Age,LoanDuration,LoanAmount,LoanRateAsPercentOfIncome,YearsAtCurrentHome,NumberOfOtherLoansAtBank,NumberOfLiableIndividuals,...,OtherLoansAtBank,OtherLoansAtStore,HasCoapplicant,HasGuarantor,OwnsHouse,RentsHouse,Unemployed,YearsAtCurrentJob_lt_1,YearsAtCurrentJob_geq_4,JobClassIsSkilled
0,1,0,1,67,6,1169,4,4,2,1,...,0,0,0,0,1,0,0,0,1,1
1,0,0,0,22,48,5951,2,2,1,1,...,0,0,0,0,1,0,0,0,0,1
2,1,0,1,49,12,2096,2,3,1,2,...,0,0,0,0,1,0,0,0,1,0
3,1,0,1,45,42,7882,2,4,1,2,...,0,0,0,1,0,0,0,0,1,1
4,1,0,1,53,24,4870,3,4,2,2,...,0,0,0,0,0,0,0,0,0,1
